<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Load-data-and--data-preprocessing" data-toc-modified-id="Load-data-and--data-preprocessing-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data and  data preprocessing</a></span></li><li><span><a href="#Event-detection-spacy" data-toc-modified-id="Event-detection-spacy-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Event detection spacy</a></span><ul class="toc-item"><li><span><a href="#Helper-functions" data-toc-modified-id="Helper-functions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Helper functions</a></span></li><li><span><a href="#Functions-for-answering-5-questions-based-on-patterns-matchs" data-toc-modified-id="Functions-for-answering-5-questions-based-on-patterns-matchs-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Functions for answering 5 questions based on patterns matchs</a></span></li><li><span><a href="#Functon-for-event-detection" data-toc-modified-id="Functon-for-event-detection-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Functon for event detection</a></span></li><li><span><a href="#Functions-for-answering-5-questions-based-on-POS,-dependencies" data-toc-modified-id="Functions-for-answering-5-questions-based-on-POS,-dependencies-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>Functions for answering 5 questions based on POS, dependencies</a></span></li><li><span><a href="#Test-functionality-on-examples" data-toc-modified-id="Test-functionality-on-examples-3.5"><span class="toc-item-num">3.5&nbsp;&nbsp;</span>Test functionality on examples</a></span></li><li><span><a href="#5-questions-and-event-detection-in-chapter-2" data-toc-modified-id="5-questions-and-event-detection-in-chapter-2-3.6"><span class="toc-item-num">3.6&nbsp;&nbsp;</span>5 questions and event detection in chapter 2</a></span></li><li><span><a href="#Total-number-of-events" data-toc-modified-id="Total-number-of-events-3.7"><span class="toc-item-num">3.7&nbsp;&nbsp;</span>Total number of events</a></span></li></ul></li><li><span><a href="#Event-detection-Giveme5w1h" data-toc-modified-id="Event-detection-Giveme5w1h-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Event detection Giveme5w1h</a></span><ul class="toc-item"><li><span><a href="#Import-libraries" data-toc-modified-id="Import-libraries-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Import libraries</a></span></li><li><span><a href="#Event-detection-in-the-chapter-2" data-toc-modified-id="Event-detection-in-the-chapter-2-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Event detection in the chapter 2</a></span></li><li><span><a href="#Total-number-of-events" data-toc-modified-id="Total-number-of-events-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Total number of events</a></span></li></ul></li></ul></div>

## Import libraries

In [1]:
import spacy
import nltk
import re
from spacy.matcher import Matcher
from spacy.util import filter_spans
from spacy import displacy
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
nlp = spacy.load('en_core_web_sm')

## Load data and  data preprocessing

In [3]:
fileObject = open("Data_pipi_longstocking/02thing.txt", "r")
data = fileObject.read()
print(data[:600])

Annika woke up early the next morning. She jumped out of bed and ran over to Tommy.
"Wake up, Tommy," she cried, pulling him by the arm. "Wake up and let's go and see that funny girl with the big shoes."
Tommy was wide-awake in an instant.
"I knew, even while I was sleeping, that something exciting was going to happen today, but I didn't remember what it was," he said as he yanked off his pajama jacket. Off they went to the bathroom, washed themselves and brushed their teeth much faster than usual, had their clothes on in a twinkling, and a whole hour before their mother expected them came sli


In [4]:
len(data)

13002

In [5]:
def remove_characters_before_tokenization(sentence, keep_apostrophes=False):
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[?|$|&|*|%|@|(|)|~]' # add other characters here to remove them
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    else:
        PATTERN = r'[^a-zA-Z0-9" ]' # only extract alpha-numeric characters
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    return filtered_sentence


In [6]:
sentences = data.split('.')
print(sentences[:5])
filtered_sentences = []
for sentence in sentences:
    filtered_sentences.append(remove_characters_before_tokenization(sentence, keep_apostrophes=False ))
print(filtered_sentences[:5])

['Annika woke up early the next morning', ' She jumped out of bed and ran over to Tommy', '\n"Wake up, Tommy," she cried, pulling him by the arm', ' "Wake up and let\'s go and see that funny girl with the big shoes', '"\nTommy was wide-awake in an instant']
['Annika woke up early the next morning', 'She jumped out of bed and ran over to Tommy', '"Wake up Tommy" she cried pulling him by the arm', '"Wake up and lets go and see that funny girl with the big shoes', '"Tommy was wideawake in an instant']


In [7]:
filtered_text = '. '.join(sent for sent in filtered_sentences)

In [8]:
filtered_text[:500]

'Annika woke up early the next morning. She jumped out of bed and ran over to Tommy. "Wake up Tommy" she cried pulling him by the arm. "Wake up and lets go and see that funny girl with the big shoes. "Tommy was wideawake in an instant. "I knew even while I was sleeping that something exciting was going to happen today but I didnt remember what it was" he said as he yanked off his pajama jacket. Off they went to the bathroom washed themselves and brushed their teeth much faster than usual had thei'

## Event detection spacy

### Helper functions

In [9]:
def show_pos_dep(doc):
    """
    Function provides information about POS tags, dependency tags, entity labels and head text and labels 
    """
    column1 = "token text"
    column2 = "token pos"
    column3 = "token dep"
    column4 = "entity label"
    column5 = "head text"
    column6 = "head pos"
    print(f"{column1:<12}{column2:<12}" \
              f"{column3:<12}{column4:<12}{column5:<12}{column6:<12}")
    for token in doc:
        token_text = token.text
        token_pos = token.pos_
        token_dep = token.dep_
        token_label = token.ent_type_
        token_head = token.head.text
        token_head_pos = token.head.pos_
        print(f"{token_text:<12}{token_pos:<12}" \
              f"{token_dep:<12}{token_label:<12}{token_head:<12}{token_head_pos:<12}")

###  Functions for answering 5 questions based on patterns matchs

In [10]:
def show_dep_tree(doc):
    """
    Function shows dependency tree
    """
    displacy.render(doc, style='dep', jupyter=True)

In [11]:
def match_by_pattern(doc, pattern):
    """
    Function extracts phrases by given pattern
    """
    matched_span = []
    matcher = Matcher(nlp.vocab)
    matcher.add("match_by_pattern", [pattern])
    matches = matcher(doc)
    for match_id, start, end in matches:
        matched_span.append(doc[start:end])
    
    return filter_spans(matched_span)

In [12]:
# patterns for phrase extraction:
pattern_who = [
    {"POS": {"REGEX": "(NOUN|PROPN|PRON)"},"DEP": "nsubj", "OP": "*"}
]

{"REGEX": "(fifa|icc)"}
pattern_what = [
    {"POS": "VERB","DEP": {"REGEX": "(ROOT|conj)"}, "OP": "+"},
    {"POS": "ADP", "DEP": {"REGEX": "(advmod|prt)"}, "OP": "?"}
]

pattern_whom = [
    {"POS": {"REGEX": "(NOUN|PROPN|PRON)"},"DEP":  {"REGEX": "dobj|pobj"}}
]
pattern_whom_subj = [
    {"POS": {"REGEX": "(NOUN|PROPN|PRON)"},"DEP": "nsubj", "HEAD.POS":{"REGEX": "ADP|AUX"}}
]

pattern_VP = [{'POS': 'VERB', 'OP': '?'},
           {'POS': 'ADV', 'OP': '*'},
           {'POS': 'VERB', 'OP': '+'}]


### Functon for event detection

In [13]:
def count_events(doc):
    """
    Function counts events in the given doc based on pattern
    """
    events_count = 0
    events_list = match_by_pattern(doc, pattern_VP)
            
    return events_list

### Functions for answering 5 questions based on POS, dependencies

In [14]:
def who_entity(doc):
    who = []
    entity_who_list = ['PERSON', 'ORG', 'NORP']
    for entity in doc.ents:
        if (entity.label_ in entity_who_list and entity.head.pos_ not in ['AUX','ADP']):
            who.append(entity.text)

    return who

def who(doc):
    who = []
    who_pos = ['NOUN','PROPN','PRON']
    head_pos = ['ADP','AUX']
    for token in doc:
        if token.pos_ in who_pos and token.dep_ == 'nsubj' and token.head.pos_ not in head_pos:
            who.append(token.text)
        if token.pos_ in who_pos and token.dep_ == 'nsubj' and token.head.pos_=='AUX' and token.head.lemma_ == 'be':
            who.append(token.text)       
    return list(dict.fromkeys(who))

In [15]:
def whom(doc):
    whom = []
    whom_pos = ['NOUN','PROPN','PRON']
    whom_dep = ['dobj','pobj']
    entity_who_list = ['PERSON', 'ORG', 'NORP']
    head_pos = ['AUX']
    for token in doc:
        if token.pos_ in whom_pos and token.dep_ in whom_dep and token.head.pos_ in head_pos:
            whom.append(token.text)
        if token.pos_ in whom_pos and token.dep_ in whom_dep and token.ent_type_ in entity_who_list:
            whom.append(token.text)
        if token.pos_ in whom_pos and token.dep_ == 'nsubj' and token.head.pos_ in head_pos and token.head.lemma_ != 'be':
            whom.append(token.text)
            
    return list(dict.fromkeys(whom))

In [16]:
def when(doc):
    when = []
    entity_when_list = ['DATE', 'TIME']
    for entity in doc.ents:
        if (entity.label_ in entity_when_list):
            when.append(entity.text)
            
    return list(dict.fromkeys(when))

In [17]:
def where (doc):
    where = []
    where_pos = ['NOUN','PROPN','PRON']
    entity_where_list = ['LOC', 'GPE']
    entity_when_list = ['DATE', 'TIME']
    entity_who_list = ['PERSON']

    for token in doc:
        if token.pos_ in where_pos and token.dep_ in ['nsubj','pobj'] and token.head.pos_ == 'ADP' and token.ent_type_ not in entity_when_list + entity_who_list:
            where.append(token.text)
        elif token.ent_type_ in entity_where_list:
            where.append(token.text)
    return list(dict.fromkeys(where))

In [18]:
def what (doc):
    what = [] 
    for j,token in enumerate(doc):
        if (token.pos_ =='VERB') and (token.dep_ in ['ROOT','conj']):
            current_what = token.text
            if j<(len(doc)-1):
                if doc[j+1].pos_ == 'ADP' and doc[j+1].dep_ in ['advmod','prt']:
                    current_what = current_what + ' ' + doc[j+1].text
            what.append(current_what)

    return list(dict.fromkeys(what))

In [19]:
def print_5w(doc):
    """
    Function prints answers for 5 questions for single document
    """
    column1 = "who"
    column2 = "did what"
    column3 = "when"
    column4 = "where"
    column5 = "to whom"
    
    #answer_who = ','.join(map(str,match_by_pattern(doc, pattern_who)))
    answer_who = ','.join(map(str,who(doc)))
    #answer_what = ','.join(map(str,match_by_pattern(doc, pattern_what)))
    answer_what = ','.join(map(str,what(doc)))
    answer_when = ','.join(map(str,when(doc)))
    answer_where = ','.join(map(str,where(doc)))
    #answer_whom = ','.join(map(str,match_by_pattern(doc, pattern_whom)))
    answer_whom = ','.join(map(str,whom(doc)))   
    print(f"{column1:<25}{column2:<25}" \
              f"{column3:<25}{column4:<25}{column5:<25}")
    print(f"{answer_who:<25}{answer_what:<25}{answer_when:<25}" \
              f"{answer_where:<25}{answer_whom:<25}")

In [20]:
def print_5w_pd(text):
    """
    Function generates Pandas DataFrame that contains answers for 5 questions for the whole text sentencewise
    """
    columns = ['sentence','who','what','when','where','whom','count_events','event_desc']
    df = pd.DataFrame(columns = columns)
    for sentence in text:
        doc = nlp(sentence)
        answer_who = ','.join(map(str,who(doc)))
        answer_what = ','.join(map(str,what(doc)))
        answer_when = ','.join(map(str,when(doc)))
        answer_where = ','.join(map(str,where(doc)))
        answer_whom = ','.join(map(str,whom(doc)))  
        event_desc = count_events(doc)
        count_events_sent = len(event_desc)
        row = [sentence, answer_who, answer_what,answer_when,answer_where,answer_whom, count_events_sent, event_desc]
        df.loc[len(df)] = row
    
    return df
    

### Test functionality on examples

In [21]:
test = filtered_sentences[2]
doc = nlp(test)
print(test, '\n')
print_5w(doc)
print('\n')
show_pos_dep(doc)
events = count_events(doc)
print('\n')
print("Num of events:",len(events),". Events:", events)

"Wake up Tommy" she cried pulling him by the arm 

who                      did what                 when                     where                    to whom                  
she                      cried                                             arm                      Tommy                    


token text  token pos   token dep   entity labelhead text   head pos    
"           PUNCT       ROOT                    "           PUNCT       
Wake        VERB        advcl                   cried       VERB        
up          ADP         prt                     Wake        VERB        
Tommy       PROPN       dobj        PERSON      Wake        VERB        
"           PUNCT       punct                   cried       VERB        
she         PRON        nsubj                   cried       VERB        
cried       VERB        ROOT                    cried       VERB        
pulling     VERB        xcomp                   cried       VERB        
him         PRON        dobj          

In [22]:
show_dep_tree(doc)

In [23]:
test = filtered_sentences[6]
#test = 'He was at the station and she went to the bathroom and gave key to the sister in the morning on Monday'
doc = nlp(test)
print(test, '\n')
print_5w(doc)
print('\n')
show_pos_dep(doc)
events = count_events(doc)
print('\n')
print("Num of events:",len(events),". Events:", events)

Off they went to the bathroom washed themselves and brushed their teeth much faster than usual had their clothes on in a twinkling and a whole hour before their mother expected them came sliding down the banister and landed at the breakfast table 

who                      did what                 when                     where                    to whom                  
they,mother              went,brushed,had,landed  a whole hour             bathroom,clothes,twinkling,banister,tablethem                     


token text  token pos   token dep   entity labelhead text   head pos    
Off         INTJ        advmod                  went        VERB        
they        PRON        nsubj                   went        VERB        
went        VERB        ROOT                    went        VERB        
to          ADP         prep                    went        VERB        
the         DET         det                     bathroom    NOUN        
bathroom    NOUN        pobj               

In [24]:
show_dep_tree(doc)

### 5 questions and event detection in chapter 2

In [25]:
df_5w = print_5w_pd(filtered_sentences)

In [26]:
df_5w.head(30)

,sentence,who,what,when,where,whom,count_events,event_desc
0,Annika woke up early the next morning,Annika,woke up,early the next morning,,,1,[(woke)]
1,She jumped out of bed and ran over to Tommy,She,"jumped,ran over",,bed,Tommy,2,"[(jumped), (ran)]"
2,"""Wake up Tommy"" she cried pulling him by the arm",she,cried,,arm,Tommy,2,"[(Wake), (cried, pulling)]"
3,"""Wake up and lets go and see that funny girl with the big shoes",,"Wake up,go,see",,shoes,,3,"[(Wake), (go), (see)]"
4,"""Tommy was wideawake in an instant",Tommy,,,instant,,0,[]
5,"""I knew even while I was sleeping that something exciting was going to happen today but I didnt remember what it was"" he said as he yanked off his pajama jacket","I,something,it,he","knew,remember",today,,,7,"[(knew), (sleeping), (going), (happen), (remember), (said), (yanked)]"
6,Off they went to the bathroom washed themselves and brushed their teeth much faster than usual had their clothes on in a twinkling and a whole hour before their mother expected them came sliding down the banister and landed at the breakfast table,"they,mother","went,brushed,had,landed",a whole hour,"bathroom,clothes,twinkling,banister,table",them,7,"[(went), (washed), (brushed), (had), (expected), (sliding), (landed)]"
7,Down they sat and announced that they wanted their hot chocolate right at that very moment,"they,chocolate","sat,announced",,moment,,3,"[(sat), (announced), (wanted)]"
8,"""Whats going to happen today that youre in such a hurry"" asked their mother","What,you",asked,today,hurry,,4,"[(going), (happen), (re), (asked)]"
9,"""Were going to see the new girl next door"" said Tommy",Tommy,said,,,,3,"[(going), (see), (said)]"


### Total number of events

In [27]:
print("Total number of events in the chapter:")
print(df_5w["count_events"].sum(axis = 0))

Total number of events in the chapter:
431


In [28]:
df_5w.to_csv('5w.csv', sep = '|')

## Event detection Giveme5w1h

### Import libraries

In [29]:
from stanfordcorenlp import StanfordCoreNLP
#nlp = StanfordCoreNLP(r'C:\ProgramData\Anaconda3\Lib\site-packages\Giveme5W1H\runtime-resources\stanford-corenlp-full-2017-06-09')
nlp = StanfordCoreNLP('http://localhost', port=9000)

In [30]:
import geopy
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = "application1"
geopy.geocoders.options.default_timeout = 1000
geolocator = Nominatim()
print(geolocator.headers)
{'User-Agent': 'my_app/1'}
print(geolocator.timeout)

{'User-Agent': 'application1'}
1000


In [31]:
from Giveme5W1H.extractor.document import Document
from Giveme5W1H.extractor.extractor import MasterExtractor

In [32]:
filtered_text = '. '.join(sent for sent in filtered_sentences)

### Event detection in the chapter 2

In [33]:
#extractor = MasterExtractor()

#date_publish = '2021-07-01 09:45:00'
#doc = Document.from_text(filtered_text)

#doc = extractor.parse(doc)

In [34]:
def Giveme5W1H_top_answers(doc):
    top_who_answer = ''
    top_what_answer = ''
    top_when_answer = ''
    top_where_answer = ''
    top_why_answer = ''
    try:
        top_who_answer = doc.get_top_answer('who').get_parts_as_text()
    except:
        pass
    try:
        top_what_answer = doc.get_top_answer('what').get_parts_as_text()
    except:
        pass
    try:
        top_when_answer = doc.get_top_answer('when').get_parts_as_text()
    except:
        pass
    try:
        top_where_answer = doc.get_top_answer('where').get_parts_as_text()
    except:
        pass
    try:
        top_why_answer = doc.get_top_answer('why').get_parts_as_text()
    except:
        pass

    return [top_who_answer, top_what_answer, top_when_answer , top_where_answer, top_why_answer  ]



In [35]:
def Giveme5W1H_answers(doc):
    top_who_answer = ''
    top_what_answer = ''
    top_when_answer = ''
    top_where_answer = ''
    top_why_answer = ''
    try:
        top_who_answer = doc.get_answers('who').get_parts_as_text()
    except:
        pass
    try:
        top_what_answer = doc.get_answers('what').get_parts_as_text()
    except:
        pass
    try:
        top_when_answer = doc.get_answers('when').get_parts_as_text()
    except:
        pass
    try:
        top_where_answer = doc.get_answers('where').get_parts_as_text()
    except:
        pass
    try:
        top_why_answer = doc.get_answers('why').get_parts_as_text()
    except:
        pass

    return [top_who_answer, top_what_answer, top_when_answer , top_where_answer, top_why_answer]

In [36]:
def print_Giveme5W1H_pd(text):
    """
    Function generates Pandas DataFrame that contains answers for 5 questions for the whole text sentencewise
    """
    extractor = MasterExtractor()
    columns = ['sentence','who','what','when','where','why']
    df = pd.DataFrame(columns = columns)
    exeptions = [
        'I remember once when I was out hunting for things in the jungles of Borneo',
        '"Tommy and Annika looked at Pippi to see just how a ThingFinder acted',
        'What a babe"He slapped his knees and laughed and in an instant they had all docked around Pippi all except Willie who wiped away his tears and walked cautiously over to stand beside Tommy',
        '" He took hold of one of Pippis braids but dropped it instantly and cried "Ouch I burned myself',
        '"Then all five boys joined hands around Pippi jumping up and down and screaming "Redhead Redhead"Pippi stood in the middle of the ring and smiled in the friendliest way',
        'When they were back home in Pippis garden Pippi said "Dear me how awful Here I found two beautiful things and you didnt get anything',
        '"Thats the way they sleep in Guatemala" she announced'
    ]
    for sentence in text:
        print (sentence)
        if sentence not in exeptions:
            doc = Document.from_text(sentence)
            doc = extractor.parse(doc)
            row = [sentence] +  Giveme5W1H_top_answers(doc)
            df.loc[len(df)] = row
    
    return df

In [37]:
df_Giveme5W1H = print_Giveme5W1H_pd(filtered_sentences)

Annika woke up early the next morning
She jumped out of bed and ran over to Tommy
"Wake up Tommy" she cried pulling him by the arm
"Wake up and lets go and see that funny girl with the big shoes
"Tommy was wideawake in an instant
"I knew even while I was sleeping that something exciting was going to happen today but I didnt remember what it was" he said as he yanked off his pajama jacket
Off they went to the bathroom washed themselves and brushed their teeth much faster than usual had their clothes on in a twinkling and a whole hour before their mother expected them came sliding down the banister and landed at the breakfast table
Down they sat and announced that they wanted their hot chocolate right at that very moment
"Whats going to happen today that youre in such a hurry" asked their mother
"Were going to see the new girl next door" said Tommy
"We may stay all day" said Annika
That morning Pippi was busy making pepparkakora kind of Swedish cookie
She had made an enormous amount of d

At least she ought to have looked scared
When nothing happened he gave her a push
"I dont think you have a very nice way with ladies" said Pippi
And she lifted him in her strong armshigh in the airand carried him to a birch tree and hung him over a branch
Then she took the next boy and hung him over another branch
The next one she set on a gatepost out side a cottage and the next she threw right over a fence so that he landed in a flowerbed
The last of the fighters she put in a tiny toy cart that stood by the road
Then Pippi Tommy Annika and Willie stood and looked at the boys for a while
The boys were absolutely speechless with fright
And Pippi said "You are cowards
Five of you attacking one boy Thats cowardly
Then you begin to push a helpless little girl around
Oh how mean""Come now well go home" she said to Tommy and Annika
And to Willie "If they try to hurt you again you come and tell me
" And to Bengt who sat up in the tree and didnt dare to stir she said is there anything else yo

In [38]:
df_Giveme5W1H.head(20)

,sentence,who,what,when,where,why
0,Annika woke up early the next morning,,,,,Annika
1,She jumped out of bed and ran over to Tommy,,,,,
2,"""Wake up Tommy"" she cried pulling him by the arm",,,,,
3,"""Wake up and lets go and see that funny girl with the big shoes",,,,,
4,"""Tommy was wideawake in an instant",,,,,
5,"""I knew even while I was sleeping that something exciting was going to happen today but I didnt remember what it was"" he said as he yanked off his pajama jacket",I,knew even while I,,,
6,Off they went to the bathroom washed themselves and brushed their teeth much faster than usual had their clothes on in a twinkling and a whole hour before their mother expected them came sliding down the banister and landed at the breakfast table,they,went to the bathroom and brushed their teeth,,,they
7,Down they sat and announced that they wanted their hot chocolate right at that very moment,they,sat and announced that they,,,they
8,"""Whats going to happen today that youre in such a hurry"" asked their mother",,,,,Whats
9,"""Were going to see the new girl next door"" said Tommy",,,,,


### Total number of events

In [39]:
print("Total number of events in the chapter:")
print(df_Giveme5W1H[df_Giveme5W1H["what"] !=''].what.count())

Total number of events in the chapter:
62


In [40]:
df_Giveme5W1H.to_csv('Giveme5W1H.csv', sep = '|')